# Training Pipeline for Natural Language Question to SPARQL Query

In [ ]:
#!pip install transformers evaluate torch

In [ ]:
#!pip install sacrebleu

In [1]:
import sys
import os
import random
import evaluate

import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, GPTJModel, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, pipeline
from sklearn.model_selection import train_test_split

from huggingface_hub import notebook_login

from datasets import load_dataset, Dataset

In [2]:
notebook_login()

Login successful
Your token has been saved to /home/jstil/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


## Select Model

In [3]:
selection = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(selection)
if selection=="t5-small":
    model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")
elif selection=="yazdipour/text-to-sparql-t5-base":
    model = AutoModelForSeq2SeqLM.from_pretrained("yazdipour/text-to-sparql-t5-base")
else:
    model = GPTJModel.from_pretrained("hf-internal-testing/tiny-random-gptj")

/home/jstil/.local/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:156: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


## Configure Environment

In [4]:
# set seed
random.seed(0)
np.random.seed(0)

## Import Data

In [2]:
sports_indices = {6144, 16386, 26627, 14339, 22531, 8197, 14343, 2059, 6161, 10263, 28696, 12313, 4127, 16417, 12323, 4133, 12328, 20522, 16427, 12333, 10286, 26672, 28724, 10296, 58, 28731, 10298, 61, 16446, 28739, 26692, 10309, 10310, 8262, 10314, 18508, 24652, 10318, 28755, 10324, 24660, 8276, 10328, 18521, 8285, 95, 96, 26723, 16486, 14439, 20587, 16493, 20594, 10357, 26742, 12407, 14457, 12413, 16509, 20606, 20607, 16513, 14466, 24707, 132, 26755, 10374, 10371, 12419, 16523, 4237, 28818, 6135, 6292, 4245, 12439, 22684, 8353, 26787, 20644, 10405, 22692, 12454, 16550, 10409, 16552, 6316, 28846, 26800, 12464, 22707, 16564, 181, 12470, 10425, 14527, 10432, 16576, 201, 10446, 10448, 16594, 6354, 22741, 24795, 20702, 12510, 10465, 14562, 10470, 26855, 231, 234, 24811, 4336, 6386, 28915, 242, 16631, 10488, 2298, 22780, 4349, 28924, 18685, 22785, 18690, 12548, 24839, 28936, 10505, 8458, 4363, 265, 22798, 271, 28943, 24849, 10517, 278, 10519, 26908, 12576, 10538, 28972, 16684, 4399, 14639, 10544, 10548, 8502, 26935, 20792, 24887, 10553, 10552, 10556, 4413, 16702, 16696, 24895, 10561, 321, 16718, 334, 10578, 8532, 12632, 29017, 10584, 4444, 26973, 10589, 348, 26977, 6498, 24930, 16743, 4456, 4460, 24941, 29042, 2420, 12665, 27001, 14716, 27013, 14729, 4491, 16780, 27022, 16783, 10641, 18833, 24982, 407, 16793, 4506, 27035, 24991, 417, 29090, 27042, 29089, 420, 29094, 4519, 12716, 29101, 16813, 430, 6577, 10673, 8628, 16820, 14334, 20920, 14778, 4539, 10689, 27073, 16841, 2506, 20939, 20940, 20941, 20942, 27087, 29136, 464, 29140, 18902, 14807, 27100, 16860, 10721, 25058, 20966, 23015, 14823, 10729, 490, 6636, 27117, 2543, 20976, 4593, 18930, 14836, 16885, 4598, 25077, 4606, 23039, 20992, 14852, 16901, 18948, 519, 16903, 29193, 29194, 4625, 27154, 14869, 23062, 25111, 23061, 27159, 21018, 16918, 21020, 27166, 10783, 12831, 29221, 6694, 29224, 10793, 10794, 14889, 16943, 25136, 25137, 21042, 2611, 4660, 21045, 14906, 12859, 23100, 16956, 16958, 14908, 19010, 27204, 6725, 29260, 10829, 2637, 12881, 29268, 23124, 10838, 16985, 12891, 4700, 27229, 23134, 23135, 6749, 609, 8802, 21090, 25189, 10856, 17003, 2668, 10861, 2669, 17010, 8818, 29299, 629, 14966, 4728, 10876, 17020, 29313, 10883, 17027, 17030, 23175, 2694, 17034, 17042, 15008, 27296, 4768, 29347, 2724, 25253, 4774, 17059, 6822, 19107, 15018, 19114, 29357, 25261, 17073, 4787, 23220, 25271, 19130, 25275, 699, 4796, 22527, 703, 10946, 4804, 15044, 25286, 712, 4813, 10958, 17102, 15056, 17103, 13011, 21205, 21210, 23259, 10976, 8934, 4838, 10984, 29416, 744, 23281, 15091, '0', 2807, 19193, 4858, 763, 4859, 27388, 21248, 770, 13058, 2820, 17156, 11014, 8968, 17162, 15115, 11019, 2827, 2835, 15126, 23319, 15128, 17177, 17178, 4888, 23324, 4896, 17185, 15140, 25383, 13098, 11054, 27439, 29488, 17200, 29494, 17209, 15164, 25404, 2880, 2883, 9029, 2886, 4935, 27466, 4940, 25421, 9038, 17232, 23382, 19291, 2910, 29535, 25438, 25440, 17250, 9058, 29542, 4968, 2920, 29546, 25453, 27504, 883, 2931, 15223, 25465, 4988, 17277, 29564, 11136, 15235, 9092, 27524, 11144, 15240, 21386, 27530, 29578, 2957, 29583, 7060, 17302, 21404, 11165, 2974, 23452, 17312, 21415, 23464, 19369, 2988, 15277, 2989, 5037, 29620, 17333, 2999, 25528, 25535, 3014, 25543, 23497, 11212, 3021, 5071, 7130, 9184, 993, 3042, 29667, 25570, 25573, 3046, 7140, 15336, 13290, 15340, 11246, 5103, 11249, 25587, 11252, 21495, 25592, 29689, 25594, 9208, 1019, 29693, 17405, 27646, 17408, 15361, 9218, 5122, 11268, 25605, 7169, 5121, 3071, 25609, 17418, 17419, 11274, 11277, 11280, 11281, 15379, 25619, 7196, 3101, 21534, 9247, 25633, 23587, 29733, 17447, 3112, 27687, 21543, 23598, 29743, 15413, 5174, 15415, 11317, 7225, 27706, 27711, 25666, 1091, 3140, 17478, 25671, 25673, 17483, 29773, 25678, 15439, 25680, 7249, 11344, 29782, 5208, 11352, 21595, 11356, 13405, 11358, 17505, 19561, 27757, 15472, 25713, 7288, 17530, 3197, 29824, 21633, 15490, 21635, 3204, 17541, 27781, 17543, 15496, 23685, 5259, 27787, 5262, 7310, 5264, 1171, 21655, 23704, 17560, 11417, 15511, 15516, 11423, 21664, 9378, 15524, 21670, 11431, 25770, 11435, 19633, 11444, 23732, 17589, 21689, 9403, 3260, 13505, 9410, 15560, 17609, 15563, 17613, 7375, 25807, 3287, 27863, 17625, 1245, 11486, 17632, 3297, 7394, 21728, 25829, 19686, 7400, 17643, 15597, 29934, 27890, 19699, 25846, 27896, 3321, 11519, 29954, 3334, 29959, 3335, 17673, 13575, 17671, 25868, 25873, 29971, 21780, 27923, 25876, 3351, 9496, 9497, 27929, 11543, 11548, 11565, 1325, 7474, 27955, 19762, 15668, 11575, 11576, 23869, 3391, 1347, 25926, 15687, 11592, 30024, 9544, 23881, 3404, 15688, 7502, 9546, 23886, 13650, 9564, 3422, 3423, 13669, 28008, 25962, 21866, 11628, 11629, 11626, 28013, 5488, 15732, 5493, 3448, 13691, 17789, 17790, 5501, 7552, 13697, 5503, 19843, 25993, 15755, 7564, 3472, 13713, 21907, 13717, 3478, 17818, 30108, 17825, 3489, 3491, 28066, 28072, 17836, 28076, 5551, 7600, 21940, 9653, 26039, 28091, 9665, 30153, 19915, 21964, 11725, 11726, 26070, 13784, 30171, 15835, 7644, 26078, 1499, 19934, 11753, 30188, 19958, 26104, 19962, 3578, 17918, 30206, 28166, 19976, 9736, 13834, 24078, 15886, 3601, 26129, 22035, 26131, 3606, 28185, 28192, 15905, 9763, 24100, 1573, 9768, 28201, 20010, 9771, 11824, 26160, 3635, 11831, 20025, 28220, 5693, 28222, 20032, 28225, 11845, 20040, 7753, 15946, 7755, 3660, 20045, 3665, 15953, 28244, 28249, 15962, 3673, 9824, 3682, 1639, 15976, 28264, 3690, 3691, 3692, 11886, 18031, 5747, 1653, 28281, 26235, 15998, 20099, 13959, 18056, 11913, 1675, 16012, 18060, 18062, 13977, 22171, 22177, 5797, 28331, 5806, 26287, 22194, 24244, 3765, 18101, 7863, 16055, 16056, 28350, 7872, 26308, 20166, 9928, 7882, 14027, 16074, 7885, 22224, 14033, 16082, 20182, 28374, 22243, 22246, 28391, 18152, 20205, 18160, 5873, 20215, 24312, 24313, 12023, 12032, 20228, 3847, 9992, 26380, 20238, 12047, 10006, 26393, 14107, 22301, 20254, 26399, 16172, 16176, 12082, 22324, 16181, 28472, 14146, 14151, 22344, 12114, 28500, 18262, 14166, 28502, 8024, 22366, 22369, 22371, 16227, 5989, 16230, 22376, 26474, 5996, 28526, 24433, 12148, 18292, 24438, 6009, 16250, 14201, 10109, 8063, 26498, 26507, 22415, 28559, 10129, 14225, 22421, 18327, 12184, 12185, 20380, 12190, 6050, 10146, 10148, 28581, 20394, 18349, 20400, 4021, 10166, 12215, 8121, 16319, 20415, 4037, 10182, 6087, 20424, 1991, 10185, 26565, 4050, 14292, 2010, 22491, 18398, 20452, 28649, 22510, 8175, 10224, 16372, 10229, 4087, 22521, 22522, 20476, 24574, 6143}

In [5]:
dataset = load_dataset("lc_quad")

train = dataset["train"].to_pandas()
train = train.dropna(subset=['uid', 'question', 'sparql_wikidata'])
sports_train = train[train['uid'].isin(sports_indices)]
sports_train.head()
train = Dataset.from_pandas(train)
train = train.train_test_split(test_size=0.2)

# test = dataset["test"].to_pandas()
# test = test.dropna(subset=['question', 'sparql_wikidata'])
# test = Dataset.from_pandas(test)

Found cached dataset lc_quad (/home/jstil/.cache/huggingface/datasets/lc_quad/default/2.0.0/e2a7d587b1ef77d8d8f62abfea5c012bc79d127575c031ae71369d0c28621f01)


  0%|          | 0/2 [00:00<?, ?it/s]

,NNQT_question,uid,subgraph,template_index,question,sparql_wikidata,sparql_dbpedia18,template,paraphrased_question
10,Did {Lindsey_Vonn} {participant of} {FIS Alpin...,519,boolean double one_hop right subgraph,19,Did Lindsey Vonn participate in the FIS Alpine...,ASK WHERE { wd:Q10068 wdt:P1344 wd:Q1428251 . ...,ASK { ?statement1 <http://www.w3.org/1999/02/...,Ask (ent-pred-obj1` . ent-pred-obj2),"In 1999 and 2019, was Lindsey Vonn in the FIS ..."
14,Did {Judi_Dench} {award received} {Tony Award ...,712,boolean double one_hop right subgraph,212,Did Judi Densch receive a Tony Award for best ...,ASK WHERE { wd:Q28054 wdt:P166 wd:Q640374 . wd...,ASK { ?statement1 <http://www.w3.org/1999/02/...,Ask (ent-pred-obj1` . ent-pred-obj2),Was the Tony Award for best direction of the p...
17,What is {played by} of {succeded by} of {Sleep...,15164,left-subgraph,86,Who Sleepwalking succeeded in playing Sleepwal...,SELECT ?answer WHERE { wd:Q7539946 wdt:P156 ?X...,SELECT ?answer WHERE { ?statement1 <http://www...,C RCD xD . xD RDE ?E,Who Sleepwalking succeeded in playing Sleepwal...
36,What is {musical score by} of {Missa Solemnis}...,12032,right-subgraph,178,What is the musical score by Missa Solemnis th...,SELECT ?answer WHERE { wd:Q723790 wdt:P86 ?ans...,SELECT ?answer WHERE { ?statement1 <http://www...,E REF ?F . ?F RFG G,What is the musical rating by means of Missa S...
57,What is the {meta-organization} for {regulated...,20025,simple question right,371,Which is the meta-organization that is regulat...,select distinct ?obj where { wd:Q2736 wdt:P37...,select distinct ?obj where { ?statement <http:...,<S P ?O ; ?O instanceOf Type>,The football association regulates what organi...


## Preprocess Data

In [6]:
X = "question"
Y = "sparql_wikidata"
prefix = "translate English to SPARQL: "
sacrebleu = evaluate.load("sacrebleu")

In [7]:
def preprocess_function(examples):
    inputs = [prefix + example for example in examples[X]]
    targets = [example for example in examples[Y]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

tokenized_data = train.map(preprocess_function, batched=True)

  0%|          | 0/16 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

## Helper Functions

In [8]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = sacrebleu.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

## Training Pipeline

In [9]:
# Configure Pipeline
shuffle = True

# hyperparams
buffer_size = 10000
batch_size = 32
epochs = 5
learning_rate = 5e-5
weight_decay=0.01
params = f"epochs={epochs}_lr={learning_rate}_wd={weight_decay}"

In [10]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [12]:
training_args = Seq2SeqTrainingArguments(
    output_dir=f"nl2sparql/t5",
    evaluation_strategy="epoch",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=weight_decay,
    save_total_limit=3,
    num_train_epochs=epochs,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/home/jstil/.local/lib/python3.10/site-packages/huggingface_hub/repository.py:705: FutureWarning: Creating a repository through 'clone_from' is deprecated and will be removed in v0.11.
  warnings.warn(
Cloning https://huggingface.co/jstilb/t5 into local empty directory.
Using cuda_amp half precision backend
The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: uid, sparql_dbpedia18, NNQT_question, question, paraphrased_question, template_index, sparql_wikidata, __index_level_0__, subgraph, template. If uid, sparql_dbpedia18, NNQT_question, question, paraphrased_question, template_index, sparql_wikidata, __inde

Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,No log,0.636715,2.494700,19.000000
2,1.420100,0.582108,3.440900,19.000000
3,0.690800,0.566394,3.614700,19.000000
4,0.631200,0.558675,3.734700,19.000000
5,0.607300,0.556748,3.771500,19.000000


The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: uid, sparql_dbpedia18, NNQT_question, question, paraphrased_question, template_index, sparql_wikidata, __index_level_0__, subgraph, template. If uid, sparql_dbpedia18, NNQT_question, question, paraphrased_question, template_index, sparql_wikidata, __index_level_0__, subgraph, template are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3859
  Batch size = 32
Saving model checkpoint to nl2sparql/t5/checkpoint-500
Configuration saved in nl2sparql/t5/checkpoint-500/config.json
Model weights saved in nl2sparql/t5/checkpoint-500/pytorch_model.bin
tokenizer config file saved in nl2sparql/t5/checkpoint-500/tokenizer_config.json
Special tokens file saved in nl2sparql/t5/checkpoint-500/special_tokens_map.json
Copy vocab file to nl2sparql/t5/checkpoint-500/spiece.mo

TrainOutput(global_step=2415, training_loss=0.7969982771152788, metrics={'train_runtime': 782.9324, 'train_samples_per_second': 98.559, 'train_steps_per_second': 3.085, 'total_flos': 933075456688128.0, 'train_loss': 0.7969982771152788, 'epoch': 5.0})

## Inference

In [ ]:
text = "translate English to SPARQL: What are the most common types of liver infection?"

In [ ]:
def all_subdirs_of(b='.'):
    result = []
    for d in os.listdir(b):
        bd = os.path.join(b, d)
        if os.path.isdir(bd): result.append(bd)
    return result

latest_dir = all_subdirs_of(f"nl2sparql/{selection}_{params}/")[-1]
print(latest_dir)

In [ ]:
translator = pipeline("translation", model=f"{latest_dir}")

In [ ]:
translator(text)